# Warm up
Let's start with some warm up exercises to get you familiar with the database and do some SQL querying.
Make sure to have downloaded the NBA dataset form Kaggle as described in the README.

In [ ]:
import sqlite3 as sql
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

## TODO: Use pathlib to get the path to the data directory. Path(__file__) won't work in a Jupyter notebook.
DATA_PATH = Path.cwd() / 'data/nba.sqlite'
# There seems to be a compatibility issue with the pathy library and the processor.

## Create a SQLite connection and run some SQL queries

In [ ]:
con = sql.connect(DATA_PATH)  # connect to the database
query = 'SELECT * FROM game LIMIT 10'  # write a query that selects the first 10 rows from the game table
top_10_game = pd.read_sql(query, con)  # read the query into a pandas dataframe
top_10_game.head()  # print the dataframe

In [ ]:
# let's look at the schema of the database
# TODO: sqlite_master is the name of the table you want to inspeact: Replace TABLE_NAME with the name of the table you want to inspect
query = """
    SELECT *
    FROM
    sqlite_master
"""

pd.read_sql(query, con).sort_values('name')

In [ ]:
# let's see all the tables we have in the database
# TODO: Filter for tables only
query = """
    SELECT name
    FROM sqlite_master
    WHERE type='table'
"""
pd.read_sql(query, con)

In [ ]:
# let's have a look at the columns in the game table
query = """
    PRAGMA table_info(draft_combine_stats)
"""
pd.read_sql(query, con)

In [ ]:
# let's find the earliest recorded game
query = """
    SELECT game_date
    FROM game
    ORDER BY game_date
    LIMIT 1
"""

print(pd.read_sql(query, con))

# let's find the latest recorded game
query = """
   PRAGMA table_info(game)
"""

print(pd.read_sql(query, con))


In [ ]:
# 1. Interesting question: Which NBA teams had the highest (home) win percentage over time?
# Answer: We create a "live" updated table that shows the win percentage of each team after each game.


# Drop the table if it already exists
con.execute("DROP TABLE IF EXISTS team_win_statistics")

# Create the table
query = """
CREATE TABLE team_win_statistics AS
SELECT 
    game.team_id_home AS team_id,
    team_details.nickname AS team_name,
    game.game_date AS date,
    SUM(CASE WHEN game.wl_home = 'W' THEN 1 ELSE 0 END) OVER (PARTITION BY game.team_id_home ORDER BY game.game_date) AS wins,
    SUM(CASE WHEN game.wl_home = 'L' THEN 1 ELSE 0 END) OVER (PARTITION BY game.team_id_home ORDER BY game.game_date) AS losses,
    ROUND(
        CAST(SUM(CASE WHEN game.wl_home = 'W' THEN 1 ELSE 0 END) OVER (PARTITION BY game.team_id_home ORDER BY game.game_date) AS FLOAT) /
        NULLIF(SUM(CASE WHEN game.wl_home IN ('W', 'L') THEN 1 ELSE 0 END) OVER (PARTITION BY game.team_id_home ORDER BY game.game_date), 0),
        2
    ) * 100 AS win_percentage
FROM game
JOIN team_details
ON game.team_id_home = team_details.team_id
ORDER BY game.team_id_home, game.game_date;
"""
con.execute(query)
# Read the first 10 rows of the new table
query = """
SELECT *
FROM team_win_statistics
ORDER BY date DESC
LIMIT 10
"""
pd.read_sql(query, con)


In [ ]:

# 1. Interesting question: What is the most successful team in the NBA in terms of wins over time?


# 1.a. Track all wins as a time series
con.execute("DROP TABLE IF EXISTS team_win_statistics") # Drop the table if it already exists
query = """
CREATE TABLE team_win_statistics AS
SELECT 
    game.team_id_home AS team_id,
    game.game_id AS game_id,
    game.game_date AS date,
    CASE WHEN game.wl_home = 'W' THEN 1 ELSE 0 END AS win
FROM game

UNION ALL

SELECT 
    game.team_id_away AS team_id,
    game.game_id AS game_id,
    game.game_date AS date,
    CASE WHEN game.wl_away = 'W' THEN 1 ELSE 0 END AS win
FROM game

ORDER BY team_id, date;
"""
con.execute(query)

query = """
ALTER TABLE team_win_statistics
ADD COLUMN team_name TEXT;
"""
con.execute(query)

query = """
UPDATE team_win_statistics
SET team_name = (
    SELECT team_details.nickname
    FROM team_details
    WHERE team_details.team_id = team_win_statistics.team_id
);
"""
con.execute(query)

query="""
ALTER TABLE team_win_statistics
ADD COLUMN ma_82 FLOAT;
"""

# Drop the table if it already exists
con.execute("DROP TABLE IF EXISTS team_win_statistics_with_ma")

con.execute(query)
query = """
CREATE TABLE team_win_statistics_with_ma AS
SELECT 
    team_id,
    game_id,
    date,
    win,
    team_name,
    ROUND(SUM(win) OVER (
        PARTITION BY team_name
        ORDER BY date
        ROWS BETWEEN 81 PRECEDING AND CURRENT ROW
    ) / 82.0, 3) AS ma_82
FROM team_win_statistics
ORDER BY team_id, date;
"""
con.execute(query)


# Delete all teams where the team_name is NULL
query = """
DELETE FROM team_win_statistics_with_ma
WHERE team_name IS NULL;
"""
con.execute(query)


query = """
SELECT *
FROM team_win_statistics_with_ma
WHERE team_name = 'Warriors'
ORDER BY date DESC
"""
pd.read_sql(query, con)





In [ ]:
# 1. b. Now let's show how the teams have performed over time.

# Read in the data into a pandas dataframe
query = """
SELECT *
FROM team_win_statistics_with_ma
"""
team_win_statistics = pd.read_sql(query, con)

# Drop all data before 2010
team_win_statistics['date'] = pd.to_datetime(team_win_statistics['date'])
team_win_statistics = team_win_statistics[team_win_statistics['date'] >= '2008-01-01']

# Drop all rows where team_name=None
team_win_statistics = team_win_statistics.dropna(subset=['team_name'])

# Drop all teams except for : "76ers", "Celtics", "Bulls", "Lakers",
team_win_statistics = team_win_statistics[team_win_statistics['team_name'].isin(["Warriors", "Lakers"])] 


# Save the data to a CSV file
team_win_statistics.to_csv('team_win_statistics.csv', index=False)




# Drop the first 82 rows of each group
team_win_statistics = team_win_statistics.apply(lambda x: x.iloc[82:])

print(team_win_statistics)



import matplotlib.pyplot as plt
import matplotlib.dates as mdates




# Plot the moving average over time for each team
plt.figure(figsize=(14, 8))

for team_name, data in team_win_statistics.groupby('team_name'):
    plt.plot(data['date'], data['ma_82'], label=team_name, alpha=0.7)

# Set up x-axis date formatting to avoid clutter
plt.gca().xaxis.set_major_locator(mdates.YearLocator(1))  # Set major ticks every 5 years
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  # Display only the year on x-axis

# Labels and title
plt.xlabel("Date")
plt.ylabel("Moving Average Win Percentage (MA 82)")
plt.title("Teams' Performance Over Time (MA 82)")
plt.legend(title="Team", bbox_to_anchor=(1.05, 1), loc='upper left')  # Legend outside the plot

# Make the lakers line #552583
plt.gca().get_lines()[1].set_color('#552583')
# Make the warriors line ##FFC72C
plt.gca().get_lines()[0].set_color('#FFC72C')

# Set dark background
plt.style.use('dark_background')

# Rotate x-axis labels for readability
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()






In [ ]:
# 2. How did the number of 3-pointers scored in the NBA change over time?

query = """
SELECT *
FROM game
"""
games = pd.read_sql(query, con)

query = """
PRAGMA table_info(game)
"""
# print(pd.read_sql(query, con))

# Surpress all warnings
import warnings
warnings.filterwarnings('ignore')

def three_pointer_regression(games : pd.DataFrame) -> int:
    # Drop all rows where fg3_pct_home is NULL
    games = games.dropna(subset=['home_win', 'fg3a_share'])
    # Regress the home_win column on the fg3_pct_home column
    X = games['fg3a_share']
    X = sm.add_constant(X)
    y = games['home_win']
    model = sm.OLS(y, X).fit()
    # return the coefficient of fg3_pct_home
    return model

# Delete all games before 1990
games['game_date'] = pd.to_datetime(games['game_date'])
games = games[games['game_date'] >= '1990-01-01']
# Create a new column which is 1 if wl_home is 'W' and 0 otherwise
games['home_win'] = (games['wl_home'] == 'W').astype(int)
# Define a new column fg3a_share which fga_home / fg3a_home
games['fg3a_share'] = games['fg3m_home'] / games['fg3a_home']


# Initialize an empty DataFrame for results
model_results = pd.DataFrame(columns=['year', 'fg3a_share_coefficient', 'p_value', 'r2', 'se_min', 'se_max'])

# Group by year and run the regression for each season
for year, data in games.groupby(games['game_date'].dt.year):
    data = data.dropna(subset=['home_win', 'fg3a_share'])
    model = three_pointer_regression(data)
    
    # Ensure se_min is smaller than se_max
    conf_int = model.conf_int().loc['fg3a_share']
    se_min, se_max = sorted(conf_int)
    
    # Create a temporary DataFrame for each year’s results and concatenate
    temp_df = pd.DataFrame({
        'year': [year],
        'fg3a_share_coefficient': [model.params['fg3a_share']],
        'p_value': [model.pvalues['fg3a_share']],
        'r2': [model.rsquared],
        'se_min': [se_min],
        'se_max': [se_max]
    })
    model_results = pd.concat([model_results, temp_df], ignore_index=True)

# Ensure the columns are numeric
model_results['year'] = pd.to_numeric(model_results['year'])
model_results['se_min'] = pd.to_numeric(model_results['se_min'])
model_results['se_max'] = pd.to_numeric(model_results['se_max'])

print(model_results)

# Plot the results. Use a seaborn lineplot with error bars.
plt.figure(figsize=(14, 8))
sns.lineplot(data=model_results, x='year', y='fg3a_share_coefficient')
plt.fill_between(model_results['year'], model_results['se_min'], model_results['se_max'], alpha=0.2)
plt.xlabel("Year")
plt.ylabel("Coefficient of fg3a_share")
plt.title("Effect of attempting 3-pointers on winning")
plt.show()



# Iterate over 